<a href="https://colab.research.google.com/github/akhila12345/akhila12345/blob/main/Yet_another_copy_of_Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:

!pip install transformers
!pip install datasets
!pip install evaluate
!pip install sentencepiece

In [ ]:
import os
import torch

import warnings
warnings.filterwarnings('ignore')

# if IN_COLAB:
#     root_path = '/content/drive/MyDrive/Try'
# else:
#     root_path = 'Enter local path'

use_mps = True if torch.has_mps else False
# os.chdir(root_path)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q sentencepiece

In [ ]:
from argparse import ArgumentParser

class Config(object):
    """Wrapper class for model hyperparameters."""
    def __init__(self):
        """
        Defaults
        """
        self.mode = None
        self.model_checkpoint = None
        self.inst_type = None
        self.experiment_name = None
        self.task = None
        self.output_dir = None
        self.id_tr_data_path = None
        self.id_te_data_path = None
        self.set_instruction_key = None
        self.ood_tr_data_path = None
        self.ood_te_data_path = None
        self.output_path = None
        self.sample_size = 1
        self.evaluation_strategy = None
        self.learning_rate = 5e-5
        self.per_device_train_batch_size = None
        self.per_device_eval_batch_size = None
        self.num_train_epochs = None
        self.weight_decay = 0.01
        self.warmup_ratio = 0.1
        self.save_strategy = 'no'
        self.load_best_model_at_end = False
        self.push_to_hub = False
        self.eval_accumulation_steps = 1
        self.predict_with_generate = True
        self.max_token_length = 128
        self.bos_instruction = None
        self.delim_instruction = None
        self.eos_instruction = None
        self.test_input = None
        self.parser = self.setup_parser()
        self.args = vars(self.parser.parse_args())
        self.__dict__.update(self.args)

    def setup_parser(self):
        """
        Sets up an argument parser
        :return:
        """
        parser = ArgumentParser(description='training code')
        parser.add_argument('-mode', help='train/eval/cli', type=str, required=True)
        parser.add_argument('-model_checkpoint', help='Huggingface Model Path', type=str, required=True)
        parser.add_argument('-inst_type', help='Decides if InstructABSA1 or InstructABSA2', type=int)
        parser.add_argument('-experiment_name', help='Name of experiment', type=str)
        parser.add_argument('-task', help='ate/atsc/joint', type=str)
        parser.add_argument('-output_dir', type=str)
        parser.add_argument('-id_tr_data_path', type=str)
        parser.add_argument('-id_te_data_path', type=str)
        parser.add_argument('-set_instruction_key', type=int, default=1)
        parser.add_argument('-ood_tr_data_path', type=str)
        parser.add_argument('-ood_te_data_path', type=str)
        parser.add_argument('-output_path', type=str)
        parser.add_argument('-sample_size', help='For sampling fraction of train data', default=1.0, type=float)
        parser.add_argument('-evaluation_strategy', help='no/epoch/steps', default='epoch', type=str)
        parser.add_argument('-learning_rate', help='learning rate', default=5e-5, type=float)
        parser.add_argument('-per_device_train_batch_size', help='per_device_train_batch_size', default=16, type=int)
        parser.add_argument('-per_device_eval_batch_size', help='per_device_eval_batch_size', default=16, type=int)
        parser.add_argument('-num_train_epochs', help='num_train_epochs', default=4, type=float)
        parser.add_argument('-weight_decay', help='Weight decay', default=0.01, type=float)
        parser.add_argument('-warmup_ratio', help='Warmup Ratio', default=0.1, type=float)
        parser.add_argument('-save_strategy', help='no/epoch/steps', default='no', type=str)
        parser.add_argument('-eval_accumulation_steps', help='Eval gradient accumulation steps', default=1, type=int)
        parser.add_argument('-predict_with_generate', help='Predict with generate', default=True, type=bool)
        parser.add_argument('-max_token_length', help='Sets maximum token output length', default=128, type=bool)
        parser.add_argument('-test_input', help='The input review to test', type=str)
        return parser

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
from datasets.dataset_dict import DatasetDict


class DatasetLoader:
    def __init__(self, train_df_id=None, test_df_id=None, val_df_id=None,
                 train_df_ood=None, test_df_ood=None, val_df_ood=None, sample_size = 1):

        self.train_df_id = train_df_id.sample(frac = sample_size, random_state = 1999) if train_df_id is not None else train_df_id
        self.test_df_id = test_df_id
        self.train_df_ood = train_df_ood
        self.test_df_ood = test_df_ood
        self.val_df_id = val_df_id
        self.val_df_ood = val_df_ood

    def reconstruct_strings(self, df, col):
        """
        Reconstruct strings to dictionaries when loading csv/xlsx files.
        """
        reconstructed_col = []
        for text in df[col]:
            if text != '[]' and isinstance(text, str):
                text = text.replace('[', '').replace(']', '').replace('{', '').replace('}', '').split(", '")
                req_list = []
                for idx, pair in enumerate(text):
                    splitter = ': ' if ': ' in pair else ':'
                    if idx%2==0:
                        reconstructed_dict = {}
                        reconstructed_dict[pair.split(splitter)[0].replace("'", '')] = pair.split(splitter)[1].replace("'", '')
                    else:
                        reconstructed_dict[pair.split(splitter)[0].replace("'", '')] = pair.split(splitter)[1].replace("'", '')
                        req_list.append(reconstructed_dict)
            else:
                req_list = text
            reconstructed_col.append(req_list)
        df[col] = reconstructed_col
        return df

    def extract_rowwise_aspect_polarity(self, df, on, key, min_val = None):
        """
        Create duplicate records based on number of aspect term labels in the dataset.
        Extract each aspect term for each row for reviews with muliple aspect term entries.
        Do same for polarities and create new column for the same.
        """
        try:
            df.iloc[0][on][0][key]
        except:
            df = self.reconstruct_strings(df, on)

        df['len'] = df[on].apply(lambda x: len(x))
        if min_val is not None:
            df.loc[df['len'] == 0, 'len'] = min_val
        df = df.loc[df.index.repeat(df['len'])]
        df['record_idx'] = df.groupby(df.index).cumcount()
        df['aspect'] = df[[on, 'record_idx']].apply(lambda x : (x[0][x[1]][key], x[0][x[1]]['polarity']) if len(x[0]) != 0 else ('',''), axis=1)
        df['polarity'] = df['aspect'].apply(lambda x: x[-1])
        df['aspect'] = df['aspect'].apply(lambda x: x[0])
        df = df.drop(['len', 'record_idx'], axis=1).reset_index(drop = True)
        return df

    def extract_rowwise_aspect_opinions(self, df, aspect_col, opinion_col, key, min_val = None):
        """
        Create duplicate records based on number of aspect term labels in the dataset.
        Extract each aspect term for each row for reviews with muliple aspect term entries.
        Do same for polarities and create new column for the same.
        """
        df['len'] = df[aspect_col].apply(lambda x: len(x))
        if min_val is not None:
            df.loc[df['len'] == 0, 'len'] = min_val
        df = df.loc[df.index.repeat(df['len'])]
        df['record_idx'] = df.groupby(df.index).cumcount()
        df['aspect'] = df[[aspect_col, 'record_idx']].apply(lambda x : x[0][x[1]][key] if len(x[0]) != 0 else '', axis=1)
        df['opinion_term'] = df[[opinion_col, 'record_idx']].apply(lambda x : x[0][x[1]][key] if len(x[0]) != 0 else '', axis=1)
        df['aspect'] = df['aspect'].apply(lambda x: ' '.join(x))
        df['opinion_term'] = df['opinion_term'].apply(lambda x: ' '.join(x))
        df = df.drop(['len', 'record_idx'], axis=1).reset_index(drop = True)
        return df

    def create_data_in_ate_format(self, df, key, text_col, aspect_col, bos_instruction = '',
                    eos_instruction = ''):
        """
        Prepare the data in the input format required.
        """
        if df is None:
            return
        try:
            df.iloc[0][aspect_col][0][key]
        except:
            df = self.reconstruct_strings(df, aspect_col)
        df['labels'] = df[aspect_col].apply(lambda x: ', '.join([i[key] for i in x]))
        df['text'] = df[text_col].apply(lambda x: bos_instruction + x + eos_instruction)
        return df


    def set_data_for_training_semeval(self, tokenize_function):
        """
        Create the training and test dataset as huggingface datasets format.
        """
        # Define train and test sets
        dataset_dict_id, dataset_dict_ood = {}, {}

        if self.train_df_id is not None:
            dataset_dict_id['train'] = Dataset.from_pandas(self.train_df_id)
        if self.test_df_id is not None:
            dataset_dict_id['test'] = Dataset.from_pandas(self.test_df_id)
        if self.val_df_id is not None:
            dataset_dict_id['validation'] = Dataset.from_pandas(self.val_df_id)
        if len(dataset_dict_id) > 1:
            indomain_dataset = DatasetDict(dataset_dict_id)
            indomain_tokenized_datasets = indomain_dataset.map(tokenize_function, batched=True)
        else:
            indomain_dataset = {}
            indomain_tokenized_datasets = {}

        if self.train_df_ood is not None:
            dataset_dict_ood['train'] = Dataset.from_pandas(self.train_df_ood)
        if self.test_df_ood is not None:
            dataset_dict_ood['test'] = Dataset.from_pandas(self.test_df_ood)
        if self.val_df_ood is not None:
            dataset_dict_ood['validation'] = Dataset.from_pandas(self.val_df_ood)
        if len(dataset_dict_id) > 1:
            other_domain_dataset = DatasetDict(dataset_dict_ood)
            other_domain_tokenized_dataset = other_domain_dataset.map(tokenize_function, batched=True)
        else:
            other_domain_dataset = {}
            other_domain_tokenized_dataset = {}

        return indomain_dataset, indomain_tokenized_datasets, other_domain_dataset, other_domain_tokenized_dataset


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments
from transformers import Trainer
from transformers import Seq2SeqTrainer

In [ ]:


class T5Generator:
    def __init__(self, model_checkpoint):
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
        self.data_collator = DataCollatorForSeq2Seq(self.tokenizer)
        self.device = 'cuda' if torch.has_cuda else ('mps' if torch.has_mps else 'cpu')
    def tokenize_function_inputs(self, sample):
        """
        Udf to tokenize the input dataset.
        """
        model_inputs = self.tokenizer(sample['text'], max_length=512, truncation=True)
        labels = self.tokenizer(sample["labels"], max_length=64, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    def train(self, tokenized_datasets, **kwargs):
        """
        Train the generative model.
        """
        #Set training arguments
        args = Seq2SeqTrainingArguments(
            **kwargs
        )
        # Define trainer object
        trainer = Seq2SeqTrainer(
            self.model,
            args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"] if tokenized_datasets.get("test") is not None else None,
            tokenizer=self.tokenizer,
            data_collator=self.data_collator,
        )
        print("Trainer device:", trainer.args.device)
        # Finetune the model
        torch.cuda.empty_cache()
        print('\nModel training started ....')
        trainer.train()
        # Save best model
        trainer.save_model()
        return trainer
    def get_labels(self, tokenized_dataset, batch_size = 4, max_length = 128, sample_set = 'train'):
        """
        Get the predictions from the trained model.
        """
        def collate_fn(batch):
            input_ids = [torch.tensor(example['input_ids']) for example in batch]
            input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
            return input_ids

        dataloader = DataLoader(tokenized_dataset[sample_set], batch_size=batch_size, collate_fn=collate_fn)
        predicted_output = []
        self.model.to(self.device)
        print('Model loaded to: ', self.device)

        for batch in tqdm(dataloader):
            batch = batch.to(self.device)
            output_ids = self.model.generate(batch, max_length = max_length)
            output_texts = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            for output_text in output_texts:
                predicted_output.append(output_text)
        return predicted_output
    def get_metrics(self, y_true, y_pred, is_triplet_extraction=False):
        total_pred = 0
        total_gt = 0
        tp = 0
        if not is_triplet_extraction:
            for gt, pred in zip(y_true, y_pred):
                gt_list = gt.split(', ')
                pred_list = pred.split(', ')
                total_pred+=len(pred_list)
                total_gt+=len(gt_list)
                for gt_val in gt_list:
                    for pred_val in pred_list:
                        if pred_val in gt_val or gt_val in pred_val:
                            tp+=1
                            break
        else:
            for gt, pred in zip(y_true, y_pred):
                gt_list = gt.split(', ')
                pred_list = pred.split(', ')
                total_pred+=len(pred_list)
                total_gt+=len(gt_list)
                for gt_val in gt_list:
                    gt_asp = gt_val.split(':')[0]

                    try:
                        gt_op = gt_val.split(':')[1]
                    except:
                        continue
                    try:
                        gt_sent = gt_val.split(':')[2]
                    except:
                        continue
                    for pred_val in pred_list:
                        pr_asp = pred_val.split(':')[0]
                        try:
                            pr_op = pred_val.split(':')[1]
                        except:
                            continue
                        try:
                            pr_sent = gt_val.split(':')[2]
                        except:
                            continue
                        if pr_asp in gt_asp and pr_op in gt_op and gt_sent == pr_sent:
                            tp+=1
        p = tp/total_pred
        r = tp/total_gt
        return p, r, 2*p*r/(p+r), None
class T5Classifier:
    def __init__(self, model_checkpoint):
        self.tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, force_download = True)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, force_download = True)
        self.data_collator = DataCollatorForSeq2Seq(self.tokenizer)
        self.device = 'cuda' if torch.has_cuda else ('mps' if torch.has_mps else 'cpu')
    def tokenize_function_inputs(self, sample):
        """
        Udf to tokenize the input dataset.
        """
        sample['input_ids'] = self.tokenizer(sample["text"], max_length = 512, truncation = True).input_ids
        sample['labels'] = self.tokenizer(sample["labels"], max_length = 64, truncation = True).input_ids
        return sample
    def train(self, tokenized_datasets, **kwargs):
        """
        Train the generative model.
        """
        # Set training arguments
        args = Seq2SeqTrainingArguments(
            **kwargs
            )
        # Define trainer object
        trainer = Trainer(
            self.model,
            args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"] if tokenized_datasets.get("test") is not None else None,
            tokenizer=self.tokenizer,
            data_collator = self.data_collator
        )
        print("Trainer device:", trainer.args.device)
        # Finetune the model
        torch.cuda.empty_cache()
        print('\nModel training started ....')
        trainer.train()
        # Save best model
        trainer.save_model()
        return trainer
    def get_labels(self, tokenized_dataset, batch_size = 4, sample_set = 'train'):
        """
        Get the predictions from the trained model.
        """
        def collate_fn(batch):
            input_ids = [torch.tensor(example['input_ids']) for example in batch]
            input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
            return input_ids
        dataloader = DataLoader(tokenized_dataset[sample_set], batch_size=batch_size, collate_fn=collate_fn)
        predicted_output = []
        self.model.to(self.device)
        print('Model loaded to: ', self.device)
        for batch in tqdm(dataloader):
            batch = batch.to(self.device)
            output_ids = self.model.generate(batch)
            output_texts = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            for output_text in output_texts:
                predicted_output.append(output_text)
        return predicted_output
    def get_metrics(self, y_true, y_pred):
        return precision_score(y_true, y_pred, average='macro'), recall_score(y_true, y_pred, average='macro'), \
            f1_score(y_true, y_pred, average='macro'), accuracy_score(y_true, y_pred)

In [ ]:
task_name = 'ate'
experiment_name = 'lapt2014_iabsa1'
model_checkpoint = 'allenai/tk-instruct-base-def-pos'
print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  lapt2014_iabsa1
Model output path:  ./Models/ate/allenaitk-instruct-base-def-pos-lapt2014_iabsa1


In [ ]:
import pandas as pd

In [ ]:
id_train_file_path = '/content/Laptops_Train.csv'
id_test_file_path = '/content/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)


loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id, 'term', 'raw_text', 'aspectTerms', '', '')
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'term', 'raw_text', 'aspectTerms', '', '')


In [ ]:
import torch

use_mps = True if torch.has_mps else False
t5_exp = T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"no",
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

# Training

In [ ]:
id_tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['sentenceId', 'raw_text', 'aspectTerms', 'aspectCategories', 'labels', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 3045
    })
    test: Dataset({
        features: ['sentenceId', 'raw_text', 'aspectTerms', 'aspectCategories', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 800
    })
})

In [ ]:
training_args

{'output_dir': './Models/ate/allenaitk-instruct-base-def-pos-lapt2014_iabsa1',
 'evaluation_strategy': 'no',
 'learning_rate': 5e-05,
 'lr_scheduler_type': 'cosine',
 'per_device_train_batch_size': 8,
 'per_device_eval_batch_size': 16,
 'num_train_epochs': 4,
 'weight_decay': 0.01,
 'warmup_ratio': 0.1,
 'save_strategy': 'no',
 'load_best_model_at_end': False,
 'push_to_hub': False,
 'eval_accumulation_steps': 1,
 'predict_with_generate': True,
 'use_mps_device': False}

In [ ]:
pip install accelerate -U

In [ ]:
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Step,Training Loss
500,1.309400
1000,0.298100
1500,0.218700


In [ ]:
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id, 'term', 'raw_text', 'aspectTerms', '', '')
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'term', 'raw_text', 'aspectTerms', '', '')

In [ ]:
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train',  batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Model loaded to:  cuda


100%|██████████| 191/191 [00:39<00:00,  4.88it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:10<00:00,  4.85it/s]


In [ ]:
p, r, f, k = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f)

p, r, f, k = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f)

Train Precision:  0.9479632063074901
Train Recall:  0.9213282247765007
Train F1:  0.9344559585492228
Test Precision:  0.9445564516129032
Test Recall:  0.9079457364341085
Test F1:  0.9258893280632411


## Inference

In [ ]:
# Step 1: Load the trained model
# Assuming model_out_path contains the path to your trained model
t5_exp = T5Generator(model_out_path)

# Step 2: Prepare the input text
# Replace 'your_input_sentence' with your actual input sentence
input_sentence = "The cab ride was amazing but the service was pricey."
model_input = input_sentence

# Step 3: Tokenize the input text
input_ids = t5_exp.tokenizer(model_input, return_tensors="pt").input_ids

# Step 4: Generate predictions
outputs = t5_exp.model.generate(input_ids, max_length=50)  # Adjust max_length as necessary

# Step 5: Decode and interpret the output
predicted_aspects = t5_exp.tokenizer.decode(outputs[0], skip_special_tokens=True)
print('Predicted Aspects:', predicted_aspects)


Predicted Aspects: service


In [ ]:
# Step 1: Load the trained model
# Assuming model_out_path contains the path to your trained model
t5_exp = T5Generator(model_out_path)

# Step 2: Prepare the input text
# Replace 'your_input_sentence' with your actual input sentence
input_sentence = "The food is very good but the quantity is very less"
model_input =   input_sentence

# Step 3: Tokenize the input text
input_ids = t5_exp.tokenizer(model_input, return_tensors="pt").input_ids

# Step 4: Generate predictions
outputs = t5_exp.model.generate(input_ids, max_length=50)  # Adjust max_length as necessary

# Step 5: Decode and interpret the output
predicted_aspects = t5_exp.tokenizer.decode(outputs[0], skip_special_tokens=True)
print('Predicted Aspects:', predicted_aspects)


Predicted Aspects: quantity


In [ ]:
# Step 1: Load the trained model
# Assuming model_out_path contains the path to your trained model
t5_exp = T5Generator(model_out_path)

# Step 2: Prepare the input text
# Replace 'your_input_sentence' with your actual input sentence
input_sentence = "The laptop is very good but the display is not satisfied much"
model_input =  input_sentence

# Step 3: Tokenize the input text
input_ids = t5_exp.tokenizer(model_input, return_tensors="pt").input_ids

# Step 4: Generate predictions
outputs = t5_exp.model.generate(input_ids, max_length=50)  # Adjust max_length as necessary

# Step 5: Decode and interpret the output
predicted_aspects = t5_exp.tokenizer.decode(outputs[0], skip_special_tokens=True)
print('Predicted Aspects:', predicted_aspects)


Predicted Aspects: display


## Aspect Sentiment Classification

In [ ]:
# Assuming id_tokenized_ds is a DatasetDict or a similar structure
if "train" in id_tokenized_ds:
    for i in range(5):
        print(id_tokenized_ds["train"][i])
else:
    print("Training dataset not found in id_tokenized_ds")


{'sentenceId': 2387, 'raw_text': 'Screen is awesome, battery life is good.', 'aspectTerms': [{'polarity': 'positive', 'term': 'Screen'}, {'polarity': 'positive', 'term': 'battery life'}], 'aspectCategories': "[{'category': 'noaspectcategory', 'polarity': 'none'}]", 'labels': [9937, 6, 3322, 280, 1], 'text': 'Screen is awesome, battery life is good.', '__index_level_0__': 798, 'input_ids': [9937, 19, 2879, 6, 3322, 280, 19, 207, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'sentenceId': 2833, 'raw_text': 'the laptop preformed pretty well.', 'aspectTerms': [{'polarity': 'positive', 'term': 'preformed'}], 'aspectCategories': "[{'category': 'noaspectcategory', 'polarity': 'none'}]", 'labels': [554, 10816, 1], 'text': 'the laptop preformed pretty well.', '__index_level_0__': 1813, 'input_ids': [8, 4544, 554, 10816, 1134, 168, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
{'sentenceId': 520, 'raw_text': 'My battery went bad about a year and a half after having it and it cos

In [ ]:
import pandas as pd

# Initialize a list to store the transformed data
transformed_data = []

# Iterate through each entry in your dataset
for entry in id_tokenized_ds['train']:
    sentence = entry['raw_text']
    for aspect_info in entry['aspectTerms']:
        aspect = aspect_info['term']
        sentiment = aspect_info['polarity']
        # Skip 'noaspectterm' entries
        if aspect != 'noaspectterm':
            transformed_data.append({'sentence': sentence, 'aspect': aspect, 'sentiment': sentiment})

# Create a DataFrame from the transformed data
aspect_sentiment_df = pd.DataFrame(transformed_data)


In [ ]:
aspect_sentiment_df

,sentence,aspect,sentiment
0,"Screen is awesome, battery life is good.",Screen,positive
1,"Screen is awesome, battery life is good.",battery life,positive
2,the laptop preformed pretty well.,preformed,positive
3,My battery went bad about a year and a half af...,battery,negative
4,This was the 3rd day and part still had not be...,shipped,negative
...,...,...,...
2353,The magnetic plug-in power charging power cord...,magnetic plug-in power charging power cord,positive
2354,The mouse is a little bit different than what ...,mouse,conflict
2355,"The design is awesome, quality is unprecedented.",design,positive
2356,"The design is awesome, quality is unprecedented.",quality,positive


In [ ]:
import pandas as pd

# Initialize a list to store the transformed data
transformed_data_test = []

# Iterate through each entry in your dataset
for entry in id_tokenized_ds['test']:
    sentence = entry['raw_text']
    for aspect_info in entry['aspectTerms']:
        aspect = aspect_info['term']
        sentiment = aspect_info['polarity']
        # Skip 'noaspectterm' entries
        if aspect != 'noaspectterm':
            transformed_data_test.append({'sentence': sentence, 'aspect': aspect, 'sentiment': sentiment})

# Create a DataFrame from the transformed data
aspect_sentiment_df_test = pd.DataFrame(transformed_data_test)


In [ ]:
aspect_sentiment_df_test

,sentence,aspect,sentiment
0,"Boot time is super fast, around anywhere from ...",Boot time,positive
1,tech support would not fix the problem unless ...,tech support,negative
2,Set up was easy.,Set up,positive
3,Did not enjoy the new Windows 8 and touchscree...,Windows 8,negative
4,Did not enjoy the new Windows 8 and touchscree...,touchscreen functions,negative
...,...,...,...
649,I've had it for about 2 months now and found n...,software,neutral
650,I've had it for about 2 months now and found n...,updates,neutral
651,the latest version does not have a disc drive.,disc drive,neutral
652,Screen - although some people might complain a...,Screen,positive


In [ ]:
from transformers import AutoTokenizer

def tokenize_aspect_sentiment_data(examples):
    tokenizer = AutoTokenizer.from_pretrained("allenai/tk-instruct-base-def-pos")
    model_inputs = tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=512)
    labels = tokenizer(examples['aspect'] + " " + examples['sentiment'], padding='max_length', truncation=True, max_length=512)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Assuming aspect_sentiment_df is your DataFrame from Step 1
tokenized_data = aspect_sentiment_df.apply(tokenize_aspect_sentiment_data, axis=1)


In [ ]:
tokenized_data_test = aspect_sentiment_df_test.apply(tokenize_aspect_sentiment_data, axis=1)


In [ ]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments


In [ ]:

# Load pre-trained model
model = T5ForConditionalGeneration.from_pretrained("allenai/tk-instruct-base-def-pos")

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
        gradient_accumulation_steps=2,  # Add this line
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data_test
)

# Train the model
trainer.train()


Step,Training Loss
10,26.917200
20,26.757500
30,26.830500
40,25.987500
50,24.680300
60,23.853300
70,22.892900
80,22.006000
90,20.614300
100,19.635100


TrainOutput(global_step=885, training_loss=4.024613315479483, metrics={'train_runtime': 507.9899, 'train_samples_per_second': 13.925, 'train_steps_per_second': 1.742, 'total_flos': 4843506200739840.0, 'train_loss': 4.024613315479483, 'epoch': 3.0})

In [ ]:
# Check the first few entries of the dataset
print(tokenized_data_test[0])

# If 'tokenized_data_test' is a Pandas DataFrame, use:
print(tokenized_data_test.head())


{'input_ids': [10722, 97, 19, 1355, 1006, 6, 300, 3461, 45, 3097, 3978, 12, 209, 1962, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# Save the model
model.save_pretrained("./AspectSentimentClassifier")

In [ ]:
from transformers import T5ForConditionalGeneration

# Load the trained model
model = T5ForConditionalGeneration.from_pretrained("./AspectSentimentClassifier")


In [ ]:
def predict_sentiment(test_dataset, model, tokenizer, device="cuda"):
    model.to(device)
    predictions = []

    for item in test_dataset:
        input_ids = torch.tensor([item['input_ids']]).to(device)
        attention_mask = torch.tensor([item['attention_mask']]).to(device)

        with torch.no_grad():
            output = model.generate(input_ids, attention_mask=attention_mask, max_length=50)

        prediction = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(prediction)

    return predictions

# Run inference
tokenizer = AutoTokenizer.from_pretrained("allenai/tk-instruct-base-def-pos")
device = "cuda" if torch.cuda.is_available() else "cpu"
predictions = predict_sentiment(tokenized_data_test, model, tokenizer, device)


In [ ]:
tokenized_data_test

0      [input_ids, attention_mask, labels]
1      [input_ids, attention_mask, labels]
2      [input_ids, attention_mask, labels]
3      [input_ids, attention_mask, labels]
4      [input_ids, attention_mask, labels]
                      ...                 
649    [input_ids, attention_mask, labels]
650    [input_ids, attention_mask, labels]
651    [input_ids, attention_mask, labels]
652    [input_ids, attention_mask, labels]
653    [input_ids, attention_mask, labels]
Length: 654, dtype: object

In [ ]:
def predict_sentiment(test_data, model, device="cuda"):
    model.to(device)
    predictions = []

    for i in range(len(test_data)):
        item = test_data.iloc[i]
        input_ids = torch.tensor([item['input_ids']], device=device)
        attention_mask = torch.tensor([item['attention_mask']], device=device)

        with torch.no_grad():
            output = model.generate(input_ids, attention_mask=attention_mask, max_length=50)

        prediction = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(prediction)

    return predictions

# Run inference
predictions = predict_sentiment(tokenized_data_test, model, device)

for i, prediction in enumerate(predictions[:5]):
    print(f"Entry {i}: Predicted Sentiment - {prediction}")


Entry 0: Predicted Sentiment - Boot time positive
Entry 1: Predicted Sentiment - tech support negative
Entry 2: Predicted Sentiment - Set up positive
Entry 3: Predicted Sentiment - Windows 8 negative
Entry 4: Predicted Sentiment - Windows 8 negative


In [ ]:
new_sentence = "The battery life of this laptop is incredible, but the keyboard is hard to use."
# Step 1: Load the trained model
# Assuming model_out_path contains the path to your trained model
t5_exp = T5Generator(model_out_path)

# Step 2: Prepare the input text
# Replace 'your_input_sentence' with your actual input sentence
model_input =  new_sentence

# Step 3: Tokenize the input text
input_ids = t5_exp.tokenizer(model_input, return_tensors="pt").input_ids

# Step 4: Generate predictions
outputs = t5_exp.model.generate(input_ids, max_length=50)  # Adjust max_length as necessary

# Step 5: Decode and interpret the output
predicted_aspects = t5_exp.tokenizer.decode(outputs[0], skip_special_tokens=True)
print('Predicted Aspects:', predicted_aspects)
# Splitting the predicted aspects into a list
aspects = predicted_aspects.split(", ")

from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch

tokenizer = AutoTokenizer.from_pretrained("allenai/tk-instruct-base-def-pos")
tokenized_inputs = []

for aspect in aspects:
    combined_input = f"{new_sentence} [aspect] {aspect}"
    inputs = tokenizer(combined_input, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
    tokenized_inputs.append(inputs)

# Load the trained model
model = T5ForConditionalGeneration.from_pretrained("./AspectSentimentClassifier")
model.eval()  # Set the model to evaluation mode
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

predicted_sentiments = []

for inputs in tokenized_inputs:
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=50)

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predicted_sentiments.append(prediction)

for aspect, sentiment in zip(aspects, predicted_sentiments):
    print(f"Aspect: {aspect}, Predicted Sentiment: {sentiment}")

Predicted Aspects: battery life, keyboard
Aspect: battery life, Predicted Sentiment: keyboard negative
Aspect: keyboard, Predicted Sentiment: battery life negative


 ## Using Predefined Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined")
model = AutoModelForSeq2SeqLM.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined")

bos_instruction = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm:none.
    Positive example 1-
    input: I charge it at night and skip taking the cord with me because of the good battery life.
    output: battery life:positive,
    Positive example 2-
    input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!.
    output: features:positive, iChat:positive, Photobooth:positive, garage band:positive
    Negative example 1-
    input: Speaking of the browser, it too has problems.
    output: browser:negative
    Negative example 2-
    input: The keyboard is too slick.
    output: keyboard:negative
    Neutral example 1-
    input: I took it back for an Asus and same thing- blue screen which required me to remove the battery to reset.
    output: battery:neutral
    Neutral example 2-
    input: Nightly my computer defrags itself and runs a virus scan.
    output: virus scan:neutral
    Now complete the following example-
    input: """
delim_instruct = ''
eos_instruct = ' \noutput:'
text = 'The food is very good but the quantity is very less'

tokenized_text = tokenizer(bos_instruction + text + delim_instruct + eos_instruct, return_tensors="pt")
output = model.generate(tokenized_text.input_ids)
print('Model output: ', tokenizer.decode(output[0], skip_special_tokens=True))

tokenizer_config.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Model output:  food:positive, quantity:negative


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined")
model = AutoModelForSeq2SeqLM.from_pretrained("kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined")

bos_instruction = """Definition: The output will be the aspects (both implicit and explicit) and the aspects sentiment polarity. In cases where there are no aspects the output should be noaspectterm:none.
    Positive example 1-
    input: I charge it at night and skip taking the cord with me because of the good battery life.
    output: battery life:positive,
    Positive example 2-
    input: I even got my teenage son one, because of the features that it offers, like, iChat, Photobooth, garage band and more!.
    output: features:positive, iChat:positive, Photobooth:positive, garage band:positive
    Negative example 1-
    input: Speaking of the browser, it too has problems.
    output: browser:negative
    Negative example 2-
    input: The keyboard is too slick.
    output: keyboard:negative
    Neutral example 1-
    input: I took it back for an Asus and same thing- blue screen which required me to remove the battery to reset.
    output: battery:neutral
    Neutral example 2-
    input: Nightly my computer defrags itself and runs a virus scan.
    output: virus scan:neutral
    Now complete the following example-
    input: """
delim_instruct = ''
eos_instruct = ' \noutput:'
text = 'The food is very bad but the quantity is very less and the appetizers is awesome'

tokenized_text = tokenizer(bos_instruction + text + delim_instruct + eos_instruct, return_tensors="pt")
output = model.generate(tokenized_text.input_ids)
print('Model output: ', tokenizer.decode(output[0], skip_special_tokens=True))

Model output:  food:negative, quantity:negative, appetizers:positive


In [ ]:
# Assuming you have a list of aspects extracted from texts
extracted_aspects = ["battery life", "screen quality", "customer service"]

# Load your T5 model fine-tuned for sentiment analysis
t5_sentiment = T5Generator(model_out_path)

# Analyze sentiment for each aspect
for aspect in extracted_aspects:
    sentiment_input = f"sentiment: {aspect}"  # Format based on your model's requirement
    input_ids = t5_sentiment.tokenizer(sentiment_input, return_tensors="pt").input_ids
    outputs = t5_sentiment.model.generate(input_ids, max_length=50)
    sentiment = t5_sentiment.tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Aspect: {aspect}, Sentiment: {sentiment}")


Aspect: battery life, Sentiment: battery life
Aspect: screen quality, Sentiment: screen quality
Aspect: customer service, Sentiment: customer service
